In [2]:
#!rm -f minsearch.py
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-13 13:53:24--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: 'minsearch.py'

     0K ...                                                   100%  579K=0.006s

2024-06-13 13:53:24 (579 KB/s) - 'minsearch.py' saved [3832/3832]



In [3]:
import requests 
#import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [11]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [26]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}]
        #temperature=0.0 
    )
    
    return response.choices[0].message.content

In [13]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [28]:
query = "10*10"
llm(query)

"Sure, here's the answer:\n\n```python\n10 * 10 < end_of_turn\n```\n\nThis code evaluates the expression `10 * 10` and compares the result with the value `end_of_turn`. Since `end_of_turn` is not specified, it is assumed to be a variable representing a specific number.\n\n**Output:**\n\nTrue\n\nThis indicates that the expression is true, meaning that 100 is less than the given number."

In [42]:
query = "What's the formula for energy?"
answer2 = llm(query)

In [45]:
print(answer2)
print(len(answer2.split(' ')))

Sure. Here's the formula for energy:

**E = 1/2 * m * v^2**

where:

* **E** is the energy in joules (J)
* **m** is the mass in kilograms (kg)
* **v** is the velocity in meters per second (m/s)

This formula describes the energy of an object based on its mass and velocity.
48


In [41]:
print(answer1)

The formula for energy is:

**E = m*c^2**

where:

* **E** is the energy in joules (J)
* **m** is the mass in kilograms (kg)
* **c** is the speed of light in a vacuum squared (m/s²)

This formula relates the energy, mass, and speed of light in a system.


In [18]:
print(answer)

Sure, here's the formula for energy:

**E = K + U**

Where:

* **E** is the energy in joules (J)
* **K** is the kinetic energy in joules (J)
* **U** is the potential energy in joules (J)

**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:

**K = 1/2 * m * v^2**

**Potential energy (U)** is the energy an object possesses when it is in a position or has a specific configuration. It is calculated as the product of an object's mass and the gravitational constant (g) multiplied by the height or distance of the object from a reference point.

**Gravitational potential energy (U)** is given by the formula:

**U = mgh**

Where:

* **m** is the mass of the object in kilograms (kg)
* **g** is the acceleration due to gravity in meters per second squared (m/s^2)
* **h** is the height or distance of the object in meters (m)

The formula for energy can be used to calculate 

In [29]:
print(len(answer)) # 1122
print(len(answer.split(' '))) # 196

196


In [34]:
import tiktoken
#encoding = tiktoken.encoding_for_model("gemma:2b")
#encoding.encode(answer)


In [35]:
import tiktoken
from tiktoken import TikTokenClient

# Initialize TikTokenClient with your API key
client = TikTokenClient(api_key='ollama')

# Example response from Gemma 2B LLM model (replace with actual response)
response_text = answer

# Count tokens in the response
token_count = client.count_tokens(response_text)

print(f"Number of tokens in the response: {token_count}")


ImportError: cannot import name 'TikTokenClient' from 'tiktoken' (/home/dheeraj/miniconda3/envs/llm_env/lib/python3.12/site-packages/tiktoken/__init__.py)